# Serverless Agents on Cloud Run Workshop

Welcome! In this workshop, we will build a "Micro-Agent" system using Google Cloud Run, Vertex AI (Gemini 2.5 Flash), and Eventarc.

**Architecture:**
1. **Librarian Service**: Ingests PDFs from Cloud Storage, summarizes them using Gemini, and saves to Firestore.
2. **Guide Service**: Answers user questions using the knowledge in Firestore.

Let's get started!

## 1. Setup & Authentication

If you don't have a project yet:

1. [Create a project](https://console.cloud.google.com/projectcreate) in the Google Cloud Console.
2. Copy your `Project ID` from the project's [Settings page](https://console.cloud.google.com/iam-admin/settings).

In [ ]:
import os
from google.colab import auth
auth.authenticate_user()

# Set your Project ID here
PROJECT_ID = "your-project-id" # @param {type:"string"}
REGION = "us-central1" # @param {type:"string"}

!gcloud config set project {PROJECT_ID}
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_REGION"] = REGION

## 1.1 Clone Repository
We need to clone the repository to get the source code for our services.

In [ ]:
!git clone https://github.com/sangalo20/Severless-agents-cloudrun.git
%cd Severless-agents-cloudrun

## 2. Enable APIs
We need to enable Cloud Run, Eventarc, Vertex AI, Firestore, and Cloud Build.

In [ ]:
!gcloud services enable run.googleapis.com eventarc.googleapis.com aiplatform.googleapis.com firestore.googleapis.com cloudbuild.googleapis.com storage.googleapis.com

## 3. Create Infrastructure
We need a Cloud Storage Bucket for uploads and a Firestore database for memory.

In [ ]:
BUCKET_NAME = f"{PROJECT_ID}-knowledge-base"
!gsutil mb -l {REGION} gs://{BUCKET_NAME}
print(f"Created bucket: {BUCKET_NAME}")

# Create Firestore in Native mode (if not exists)
!gcloud firestore databases create --location={REGION} --type=firestore-native

## 4. Deploy "The Librarian" Service
This service listens for file uploads.

In [ ]:
# Build and Deploy Librarian
SERVICE_NAME_LIBRARIAN = "librarian"

!gcloud builds submit --tag gcr.io/{PROJECT_ID}/{SERVICE_NAME_LIBRARIAN} librarian/
!gcloud run deploy {SERVICE_NAME_LIBRARIAN} --image gcr.io/{PROJECT_ID}/{SERVICE_NAME_LIBRARIAN} --region {REGION} --allow-unauthenticated

## 5. Deploy "The Guide" Service
This service handles chat interactions.

In [ ]:
# Build and Deploy Guide
SERVICE_NAME_GUIDE = "guide"

!gcloud builds submit --tag gcr.io/{PROJECT_ID}/{SERVICE_NAME_GUIDE} guide/
!gcloud run deploy {SERVICE_NAME_GUIDE} --image gcr.io/{PROJECT_ID}/{SERVICE_NAME_GUIDE} --region {REGION} --allow-unauthenticated

## 6. Wire it up with Eventarc
Create a trigger so that when a file is uploaded to the bucket, the Librarian service is called.

In [ ]:
# Grant permission to the Compute Engine service account (default for Eventarc)
PROJECT_NUMBER = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
SERVICE_ACCOUNT = f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"

!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/eventarc.eventReceiver
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/run.invoker

# Create the trigger
!gcloud eventarc triggers create librarian-trigger \
  --location={REGION} \
  --destination-run-service={SERVICE_NAME_LIBRARIAN} \
  --destination-run-region={REGION} \
  --event-filters="type=google.cloud.storage.object.v1.finalized" \
  --event-filters="bucket={BUCKET_NAME}" \
  --service-account={SERVICE_ACCOUNT}

## 7. Test it!
Upload a file and chat with the agent.

In [ ]:
# 1. Upload a dummy schedule
with open("schedule.txt", "w") as f:
    f.write("DevFest Schedule:\n10:00 AM - Keynote by Google\n11:00 AM - Serverless Agents Workshop\n12:00 PM - Lunch")

!gsutil cp schedule.txt gs://{BUCKET_NAME}/schedule.txt
print("File uploaded. Waiting for Eventarc (approx 1-2 mins)...")

In [ ]:
# 2. Chat with the Guide
import requests
import time

# Get Guide URL
GUIDE_URL = !gcloud run services describe {SERVICE_NAME_GUIDE} --region {REGION} --format='value(status.url)'
GUIDE_URL = GUIDE_URL[0]

print(f"Chatting with Guide at: {GUIDE_URL}")

query = {"session_id": "test-session", "query": "What time is the Serverless Agents workshop?"}
response = requests.post(f"{GUIDE_URL}/chat", json=query)
print(response.json())